In [1]:
from dependency import parent_dir
from common.basics import *
from common.save import make_dir, save_pickle, load_pickle, save
layer1 = json.load(open('../big_data/layer1.json','r'))

In [2]:
def clean_line(line):
    '''
    Args:
        line: a string, such as food name, sentences...
    '''
    assert type(line) == str
    
    # all lowercase
    line = line.lower()
    
    # only reserve number and alphabets
    line = re.sub(r'[^a-z0-9+()/?!.,]', ' ', line)
    
    # replace things in brace
    line = re.sub(r'\([^)]*\)', '', line)
    
    # remove extra spaces
    line = re.sub(' +',' ',line).strip()
    return line

def preprocessing(layer1):
    data = []
    for i, recipe in tqdm.tqdm(enumerate(layer1)):
        processed_recipe = {'id': recipe['id'],
                            'ingredients':[]}
        field = 'ingredients'
        for line in recipe[field]:
            cleaned = clean_line(line['text'])
            if cleaned:
                processed_recipe[field].append(cleaned)  
        data.append(processed_recipe)
        
    return data

In [3]:
data = preprocessing(layer1)

1029720it [03:03, 5602.32it/s]


In [4]:
class ny_ingredients:
    '''use ny-time-parser to process the ingredients'''
    def __init__(self):
        self.ny_ingred = '../../NYtime-parser2/ingred.txt'
        self.ny_result = '../../NYtime-parser2/result.json'
        self.fields = ['ingredients']
        
    def to_ny(self, data):
        '''
        Args: 
          data: A list of recipes 
        '''
        to_write = []
        for i, v in tqdm.tqdm(enumerate(data)):
            # assing index
            for field in self.fields:
                line_ids = []
                for line in v[field]:
                    if line in to_write:
                        ny_id = to_write.index(line)
                    else:
                        ny_id = len(to_write)
                        to_write.append(line)
                    line_ids.append(ny_id)
                data[i]['ny_%s'%(field)] = line_ids

        # save the file to the folder under NYtime-parser2
        save(filename = self.ny_ingred, 
             to_write = '\n'.join(to_write),
             overwrite = True, 
             print_=True)
        return data

1. Collect all the ingredients mentioned in this dataset and save to '../../NYtime-parser2/ingred.txt'

In [ ]:
ny_ingr = ny_ingredients()
data = ny_ingr.to_ny(data)

631756it [25:14:54,  4.91it/s]

2. Execute the notebook 'Control Nytimes.ipynb' in python 2

In [ ]:
ny_result = pd.read_json(ny_ingr.ny_result)
to_write = []
for i, v in tqdm.tqdm(enumerate(data)):
    for field in ny_ingr.fields:
        temp = [ny_result.loc[ny_id]['name'] for ny_id in v['ny_%s'%(field)]]
        data[i]['ny_full_%s'%(field)] = [ny_result.loc[ny_id].to_dict() for ny_id in v['ny_%s'%(field)]]

In [9]:
save_pickle('../big_data/recipe1M_ny_2.pickle', data, overwrite=True)

time: 58.1 s
